### Process Data

### Adding new data

In [39]:
# converting new spreadsheets to 
import pandas as pd 
import openpyxl 

pd.read_excel('fwddailynexusprarequestforlibraryflooroccupancydat/OccupancyData--9Dec22-9Jan23.xlsx').to_csv('fwddailynexusprarequestforlibraryflooroccupancydat/OccupancyData--9Dec22-9Jan23', index=False)
pd.read_excel('fwddailynexusprarequestforlibraryflooroccupancydat/OccupancyData--9Jan23-17Jun23 (1).xlsx').to_csv('fwddailynexusprarequestforlibraryflooroccupancydat/OccupancyData--9Jan23-17Jun23.csv', index=False)

### Combining files

In [207]:
# rerunning to add new data 
import glob
import os

path = r'fwddailynexusprarequestforlibraryflooroccupancydat' # use your path
all_files = glob.glob(os.path.join(path, "*.csv")) 

df_from_each_file = (pd.read_csv(f,header=5) for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)
# doesn't create a list, nor does it append to one
concatenated_df.to_csv("Lib_hours.zip",compression="gzip")

In [208]:
concatenated_df["Timestamp"].tail()

242368    01/31/2022 19:00
242369    01/31/2022 20:00
242370    01/31/2022 21:00
242371    01/31/2022 22:00
242372    01/31/2022 23:00
Name: Timestamp, dtype: object

In [209]:
concatenated_df["Average Occupancy"].max()

3622

In [210]:
import pandas as pd 
selected_vars = ['Location','Timestamp','Average Occupancy','Peak Occupancy','Time Interval','Location Path']
floor = pd.read_csv("Lib_hours.zip", compression = 'gzip', usecols = selected_vars)
# converting timstamps to readable time data
floor['Timestamp'] =  pd.to_datetime(floor['Timestamp'])
# floor.drop(columns='Unamed: 0')
floor['Timestamp'].tail()

242368   2022-01-31 19:00:00
242369   2022-01-31 20:00:00
242370   2022-01-31 21:00:00
242371   2022-01-31 22:00:00
242372   2022-01-31 23:00:00
Name: Timestamp, dtype: datetime64[ns]

Adding hour of day column 

In [212]:
floor['Hour'] = pd.to_datetime(floor['Timestamp']).dt.hour 

In [213]:
floor = floor.sort_values('Timestamp', ascending=True)
floor

,Location,Timestamp,Average Occupancy,Peak Occupancy,Time Interval,Location Path,Hour
0,UCSB Library,2021-09-22 16:00:00,12,2,1hour,UCSB > UCSB Library,16
557,2nd Floor Ocean,2021-09-22 16:00:00,12,2,1hour,UCSB > UCSB Library > 2nd Floor > 2nd Floor Ocean,16
348,2nd Floor,2021-09-22 16:00:00,12,2,1hour,UCSB > UCSB Library > 2nd Floor,16
1,UCSB Library,2021-09-25 12:00:00,30,39,1hour,UCSB > UCSB Library,12
558,2nd Floor Ocean,2021-09-25 12:00:00,3,4,1hour,UCSB > UCSB Library > 2nd Floor > 2nd Floor Ocean,12
...,...,...,...,...,...,...,...
124372,4th Floor,2023-06-17 23:00:00,0,1,1hour,UCSB > UCSB Library > 4th Floor,23
132034,6th - Quiet Floor,2023-06-17 23:00:00,0,0,1hour,UCSB > UCSB Library > 6th - Quiet Floor,23
143527,Graduate Studies,2023-06-17 23:00:00,0,1,1hour,UCSB > UCSB Library > Graduate Studies,23
101395,1st Floor Mountain,2023-06-17 23:00:00,0,0,1hour,UCSB > UCSB Library > 1st Floor > 1st Floor Mo...,23


## Quarter value assignment

Quarter Dates Value Matrix

In [214]:
import numpy as np 
import pandas as pd 
# Quarter Dates Value Matrix
quarters = pd.DataFrame(np.matrix(
    [['Fall 2021', '9/19/2021 00:00', '12/10/2021 23:00'],
    ['Winter 2022', '1/3/2022 00:00', '3/18/2022 23:00'],
    ['Spring 2022', '3/28/2022 00:00', '6/3/2022 23:00'],
    ['Summer 2022', '6/20/2022 00:00', '9/11/2022 23:00'],
    ['Fall 2022', '9/18/2022 00:00', '12/9/2022 23:00'],
    ['Winter 2023', '1/9/2023 00:00', '3/24/2023 23:00'],
    ['Spring 2023', '4/3/2023 00:00', '6/17/2023 23:00']]
    ), 
columns=['Quarter','Start','Stop'])
# creating id column, 
# quarters['ID'] = [1,2,3,4,5,6,7]
quarters

,Quarter,Start,Stop
0,Fall 2021,9/19/2021 00:00,12/10/2021 23:00
1,Winter 2022,1/3/2022 00:00,3/18/2022 23:00
2,Spring 2022,3/28/2022 00:00,6/3/2022 23:00
3,Summer 2022,6/20/2022 00:00,9/11/2022 23:00
4,Fall 2022,9/18/2022 00:00,12/9/2022 23:00
5,Winter 2023,1/9/2023 00:00,3/24/2023 23:00
6,Spring 2023,4/3/2023 00:00,6/17/2023 23:00


In [215]:
# if timestamp in between start and stop, add Quarter label to column, Undefined otherwise 
# floor['Datetime'] =  pd.to_datetime(floor['Timestamp'], format='%m/%d/%Y %H:%M') 
quarters['Start'] = pd.to_datetime(quarters['Start'], format='%m/%d/%Y %H:%M') 
quarters['Stop'] = pd.to_datetime(quarters['Stop'], format='%m/%d/%Y %H:%M') 
quarters
# floor.head()

,Quarter,Start,Stop
0,Fall 2021,2021-09-19,2021-12-10 23:00:00
1,Winter 2022,2022-01-03,2022-03-18 23:00:00
2,Spring 2022,2022-03-28,2022-06-03 23:00:00
3,Summer 2022,2022-06-20,2022-09-11 23:00:00
4,Fall 2022,2022-09-18,2022-12-09 23:00:00
5,Winter 2023,2023-01-09,2023-03-24 23:00:00
6,Spring 2023,2023-04-03,2023-06-17 23:00:00


Note: issues with spring quarter assignment. Replace date assignment's not belonging to spring 23 with n/a

In [216]:
floor = floor.sort_values('Timestamp', ascending=True)
# create a new column of interval index based on the date range in quarters
intervals = pd.IntervalIndex.from_arrays(quarters['Start'], quarters['Stop'], closed='both')
# get the index of the interval for each row in floor
indexer = intervals.get_indexer(floor['Timestamp'])
# use the category column of quarters to assign the corresponding label to each row in floor
floor['Quarter'] = quarters['Quarter'].iloc[indexer].values

# resolving spring 2023 assignment issues 
not_in_range = (floor['Timestamp'] > pd.to_datetime('6/17/2023 23:00')) | (floor['Timestamp'] < pd.to_datetime('4/3/2023 00:00'))
value_condition = floor['Quarter'] == 'Spring 2023'
floor.loc[not_in_range & value_condition, 'Quarter'] = 'N/A'

# floor
floor.to_csv("libFloor.csv", index=False)

In [217]:
floor_f21 = floor[floor['Quarter'] == 'Fall 2021']
floor_f21

,Location,Timestamp,Average Occupancy,Peak Occupancy,Time Interval,Location Path,Hour,Quarter
0,UCSB Library,2021-09-22 16:00:00,12,2,1hour,UCSB > UCSB Library,16,Fall 2021
557,2nd Floor Ocean,2021-09-22 16:00:00,12,2,1hour,UCSB > UCSB Library > 2nd Floor > 2nd Floor Ocean,16,Fall 2021
348,2nd Floor,2021-09-22 16:00:00,12,2,1hour,UCSB > UCSB Library > 2nd Floor,16,Fall 2021
1,UCSB Library,2021-09-25 12:00:00,30,39,1hour,UCSB > UCSB Library,12,Fall 2021
558,2nd Floor Ocean,2021-09-25 12:00:00,3,4,1hour,UCSB > UCSB Library > 2nd Floor > 2nd Floor Ocean,12,Fall 2021
...,...,...,...,...,...,...,...,...
182534,1st Floor Ocean,2021-12-10 23:00:00,0,0,1hour,UCSB > UCSB Library > 1st Floor > 1st Floor Ocean,23,Fall 2021
181790,1st Floor Mountain,2021-12-10 23:00:00,1,2,1hour,UCSB > UCSB Library > 1st Floor > 1st Floor Mo...,23,Fall 2021
188486,7th Floor,2021-12-10 23:00:00,4,5,1hour,UCSB > UCSB Library > 7th Floor,23,Fall 2021
190716,2nd Mountain GRS,2021-12-10 23:00:00,0,0,1hour,UCSB > UCSB Library > Graduate Studies > 2nd M...,23,Fall 2021


In [218]:
floor_f22 = floor[floor['Quarter'] == 'Fall 2022']
floor_f22

,Location,Timestamp,Average Occupancy,Peak Occupancy,Time Interval,Location Path,Hour,Quarter
72715,6th Floor,2022-09-18 00:00:00,0,0,1hour,UCSB > UCSB Library > 6th Floor,0,Fall 2022
75595,2nd Mountain GRS,2022-09-18 00:00:00,0,0,1hour,UCSB > UCSB Library > Graduate Studies > 2nd M...,0,Fall 2022
73435,7th Floor,2022-09-18 00:00:00,3,21,1hour,UCSB > UCSB Library > 7th Floor,0,Fall 2022
71995,5th Floor,2022-09-18 00:00:00,58,68,1hour,UCSB > UCSB Library > 5th Floor,0,Fall 2022
71275,4th Floor,2022-09-18 00:00:00,0,1,1hour,UCSB > UCSB Library > 4th Floor,0,Fall 2022
...,...,...,...,...,...,...,...,...
64243,8th Floor,2022-12-09 23:00:00,2,2,1hour,UCSB > UCSB Library > 8th Floor,23,Fall 2022
63811,6th Floor,2022-12-09 23:00:00,10,12,1hour,UCSB > UCSB Library > 6th Floor,23,Fall 2022
65107,8th Floor GRS,2022-12-09 23:00:00,0,0,1hour,UCSB > UCSB Library > Graduate Studies > 8th F...,23,Fall 2022
62731,2nd Floor,2022-12-09 23:00:00,39,74,1hour,UCSB > UCSB Library > 2nd Floor,23,Fall 2022


In [219]:
# issues with spring 
floor_s23 = floor[floor['Quarter'] == 'Spring 2023']
floor_s23

,Location,Timestamp,Average Occupancy,Peak Occupancy,Time Interval,Location Path,Hour,Quarter
107233,Art & Architecture,2023-04-03 00:00:00,0,0,1hour,UCSB > UCSB Library > 1st Floor > Art & Archit...,0,Spring 2023
130219,6th - Quiet Floor,2023-04-03 00:00:00,0,0,1hour,UCSB > UCSB Library > 6th - Quiet Floor,0,Spring 2023
114895,2nd Floor Mountain,2023-04-03 00:00:00,0,0,1hour,UCSB > UCSB Library > 2nd Floor > 2nd Floor Mo...,0,Spring 2023
103411,1st Floor Ocean,2023-04-03 00:00:00,0,0,1hour,UCSB > UCSB Library > 1st Floor > 1st Floor Ocean,0,Spring 2023
118726,2nd Floor Ocean,2023-04-03 00:00:00,1,1,1hour,UCSB > UCSB Library > 2nd Floor > 2nd Floor Ocean,0,Spring 2023
...,...,...,...,...,...,...,...,...
112879,2nd Floor,2023-06-17 23:00:00,2,2,1hour,UCSB > UCSB Library > 2nd Floor,23,Spring 2023
128203,5th - Quiet Floor,2023-06-17 23:00:00,1,1,1hour,UCSB > UCSB Library > 5th - Quiet Floor,23,Spring 2023
154829,8th Floor GRS,2023-06-17 23:00:00,0,0,1hour,UCSB > UCSB Library > Graduate Studies > 8th F...,23,Spring 2023
147300,2nd Mountain GRS,2023-06-17 23:00:00,0,0,1hour,UCSB > UCSB Library > Graduate Studies > 2nd M...,23,Spring 2023


In [220]:
breaks = floor[floor['Quarter'] == 'N/A']
breaks

,Location,Timestamp,Average Occupancy,Peak Occupancy,Time Interval,Location Path,Hour,Quarter
186999,5th Floor,2021-12-11 00:00:00,0,0,1hour,UCSB > UCSB Library > 5th Floor,0,N/A
188487,7th Floor,2021-12-11 00:00:00,4,5,1hour,UCSB > UCSB Library > 7th Floor,0,N/A
181047,1st Floor,2021-12-11 00:00:00,0,0,1hour,UCSB > UCSB Library > 1st Floor,0,N/A
191461,2nd Ocean GRS,2021-12-11 00:00:00,0,0,1hour,UCSB > UCSB Library > Graduate Studies > 2nd O...,0,N/A
189231,8th Floor,2021-12-11 00:00:00,0,0,1hour,UCSB > UCSB Library > 8th Floor,0,N/A
...,...,...,...,...,...,...,...,...
118725,2nd Floor Ocean,2023-04-02 23:00:00,1,1,1hour,UCSB > UCSB Library > 2nd Floor > 2nd Floor Ocean,23,N/A
141711,Graduate Studies,2023-04-02 23:00:00,1,1,1hour,UCSB > UCSB Library > Graduate Studies,23,N/A
126387,5th - Quiet Floor,2023-04-02 23:00:00,0,0,1hour,UCSB > UCSB Library > 5th - Quiet Floor,23,N/A
91917,UCSB Library,2023-04-02 23:00:00,7,9,1hour,UCSB > UCSB Library,23,N/A


In [195]:
floor = pd.read_csv("libFloor.csv")
quarters = floor['Quarter'].unique()
print(quarters)

['Fall 2021' nan 'Winter 2022' 'Spring 2022' 'Summer 2022' 'Fall 2022'
 'Winter 2023' 'Spring 2023']
